# **NLP ASSIGNMENT 2:**

## **KABIR CHATURVEDI, J011**

In [1]:
import pandas as pd
import numpy as np
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

In [3]:
data = pd.read_csv('/content/Papers.csv',error_bad_lines=False, engine="python")
data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 797: unexpected end of data


,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


In [4]:
df = pd.DataFrame(data = data.paper_text, columns = ['paper_text'], index = range(len(data)))
df['index'] = range(len(data))
df.head()

,paper_text,index
0,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,0
1,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...,1
2,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,2
3,Bayesian Query Construction for Neural\nNetwor...,3
4,"Neural Network Ensembles, Cross\nValidation, a...",4


In [5]:
df.shape

(795, 2)

In [6]:
df.isnull().sum()

paper_text    0
index         0
dtype: int64

# **Now we perform preprocessing:**

In [7]:
np.random.seed(121)

In [9]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [10]:
stemmer = SnowballStemmer('english')

In [11]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos = 'v'))

def preprocess_text(text):
    result = []
    for token in simple_preprocess(text):
        if token not in STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [12]:
processed_docs = df['paper_text'].apply(preprocess_text)
processed_docs[:15]

0     [self, organ, associ, databas, applic, hisashi...
1     [mean, field, theori, layer, visual, cortex, a...
2     [store, covari, associ, long, term, potenti, d...
3     [bayesian, queri, construct, neural, network, ...
4     [neural, network, ensembl, cross, valid, activ...
5     [sing, neural, instanti, deform, model, christ...
6     [plastic, mediat, competit, learn, terrenc, se...
7     [iceg, morpholog, classif, analogu, vlsi, neur...
8     [real, time, control, tokamak, plasma, neural,...
9     [real, time, control, tokamak, plasma, neural,...
10    [learn, play, game, chess, sebastian, thrun, u...
11    [scale, data, cluster, thoma, hofmann, joachim...
12    [experiment, comparison, recurr, neural, netwo...
13    [train, multilay, perceptron, extend, kalman, ...
14    [interfer, learn, intern, model, invers, dynam...
Name: paper_text, dtype: object

# **NOW WE DO FEATURE EXTRACTION USING TF-IDF :**

In [13]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [14]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [15]:
tfidf = gensim.models.TfidfModel(bow_corpus)

In [16]:
corpus_tfidf = tfidf[bow_corpus]

# **WE RUN LDA USING TF-IDF :**

In [17]:
num_topics = 10

In [18]:
lda_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics = num_topics, id2word = dictionary, passes = 2, workers = 4)

In [19]:
for idx, topic in lda_tfidf.print_topics(-1):
    print(f"Topic : {idx} \n Words {topic} \n")

Topic : 0 
 Words 0.000*"attent" + 0.000*"bayesian" + 0.000*"imag" + 0.000*"synaps" + 0.000*"tsitsik" + 0.000*"chain" + 0.000*"motion" + 0.000*"state" + 0.000*"human" + 0.000*"train" 

Topic : 1 
 Words 0.001*"cell" + 0.000*"state" + 0.000*"neuron" + 0.000*"imag" + 0.000*"oscil" + 0.000*"rout" + 0.000*"tree" + 0.000*"analog" + 0.000*"visual" + 0.000*"polici" 

Topic : 2 
 Words 0.001*"kernel" + 0.001*"train" + 0.001*"imag" + 0.001*"spike" + 0.001*"posterior" + 0.001*"mixtur" + 0.001*"bind" + 0.001*"gaussian" + 0.001*"classifi" + 0.001*"neuron" 

Topic : 3 
 Words 0.001*"circuit" + 0.001*"neuron" + 0.000*"chip" + 0.000*"transistor" + 0.000*"endmemb" + 0.000*"match" + 0.000*"cell" + 0.000*"voltag" + 0.000*"node" + 0.000*"transit" 

Topic : 4 
 Words 0.000*"blind" + 0.000*"rotat" + 0.000*"eigenvector" + 0.000*"robot" + 0.000*"spike" + 0.000*"unit" + 0.000*"psom" + 0.000*"textur" + 0.000*"monk" + 0.000*"neuron" 

Topic : 5 
 Words 0.001*"imag" + 0.001*"tree" + 0.000*"neuron" + 0.000*"rank"

## **To find the common words b/w each pair of topics:**

In [20]:
topic_dict = {}
for idx, topic in lda_tfidf.print_topics(-1):
    topic_dict[idx] = topic

topic_dict

{0: '0.000*"attent" + 0.000*"bayesian" + 0.000*"imag" + 0.000*"synaps" + 0.000*"tsitsik" + 0.000*"chain" + 0.000*"motion" + 0.000*"state" + 0.000*"human" + 0.000*"train"',
 1: '0.001*"cell" + 0.000*"state" + 0.000*"neuron" + 0.000*"imag" + 0.000*"oscil" + 0.000*"rout" + 0.000*"tree" + 0.000*"analog" + 0.000*"visual" + 0.000*"polici"',
 2: '0.001*"kernel" + 0.001*"train" + 0.001*"imag" + 0.001*"spike" + 0.001*"posterior" + 0.001*"mixtur" + 0.001*"bind" + 0.001*"gaussian" + 0.001*"classifi" + 0.001*"neuron"',
 3: '0.001*"circuit" + 0.001*"neuron" + 0.000*"chip" + 0.000*"transistor" + 0.000*"endmemb" + 0.000*"match" + 0.000*"cell" + 0.000*"voltag" + 0.000*"node" + 0.000*"transit"',
 4: '0.000*"blind" + 0.000*"rotat" + 0.000*"eigenvector" + 0.000*"robot" + 0.000*"spike" + 0.000*"unit" + 0.000*"psom" + 0.000*"textur" + 0.000*"monk" + 0.000*"neuron"',
 5: '0.001*"imag" + 0.001*"tree" + 0.000*"neuron" + 0.000*"rank" + 0.000*"expert" + 0.000*"polici" + 0.000*"mixtur" + 0.000*"predict" + 0.000*

In [21]:
from itertools import combinations
import regex as re

In [22]:
def topic_words(topic):
    topics = re.findall("\D{4,}", topic)
    return list(map(lambda x: x[2: x.find('"', x.find('"') + 1)], topics))

In [23]:
for combination in combinations(list(range(10)), 2):
    common_words = set(topic_words(topic_dict[combination[0]])).intersection(set(topic_words(topic_dict[combination[1]])))
    print(f"Topic {combination[0]} and {combination[1]} : {len(common_words)}", end = "\n")
    print(f"Common words - {common_words}", end = "\n")
    print("\n")

Topic 0 and 1 : 2
Common words - {'state', 'imag'}


Topic 0 and 2 : 2
Common words - {'train', 'imag'}


Topic 0 and 3 : 0
Common words - set()


Topic 0 and 4 : 0
Common words - set()


Topic 0 and 5 : 1
Common words - {'imag'}


Topic 0 and 6 : 1
Common words - {'synaps'}


Topic 0 and 7 : 3
Common words - {'state', 'train', 'imag'}


Topic 0 and 8 : 0
Common words - set()


Topic 0 and 9 : 1
Common words - {'imag'}


Topic 1 and 2 : 2
Common words - {'neuron', 'imag'}


Topic 1 and 3 : 2
Common words - {'cell', 'neuron'}


Topic 1 and 4 : 1
Common words - {'neuron'}


Topic 1 and 5 : 4
Common words - {'tree', 'imag', 'neuron', 'polici'}


Topic 1 and 6 : 2
Common words - {'cell', 'neuron'}


Topic 1 and 7 : 3
Common words - {'state', 'imag', 'polici'}


Topic 1 and 8 : 1
Common words - {'neuron'}


Topic 1 and 9 : 1
Common words - {'imag'}


Topic 2 and 3 : 1
Common words - {'neuron'}


Topic 2 and 4 : 2
Common words - {'spike', 'neuron'}


Topic 2 and 5 : 3
Common words - {'mixtur

# **Evaluating the topic model:**

- **Higher the coherence score , the better it is.**

In [24]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_tfidf, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3196601303062716


### **Using a different model to get a higher coherence score -**

-   **Increasing the number of passes as well as the chunksize:**



In [25]:
lda_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics = num_topics, id2word = dictionary, passes = 15, workers = 4, 
                                       per_word_topics = True, chunksize = 3000)

In [26]:
for idx, topic in lda_tfidf.print_topics(-1):
    print(f"Topic : {idx} \n Words {topic} \n")

Topic : 0 
 Words 0.000*"lissom" + 0.000*"malcom" + 0.000*"chmms" + 0.000*"lstm" + 0.000*"meir" + 0.000*"bipe" + 0.000*"alvi" + 0.000*"yellow" + 0.000*"stensmo" + 0.000*"robin" 

Topic : 1 
 Words 0.000*"rangacv" + 0.000*"hpnn" + 0.000*"equipartit" + 0.000*"malign" + 0.000*"spert" + 0.000*"polygon" + 0.000*"minimax" + 0.000*"boxplot" + 0.000*"ih" + 0.000*"caviti" 

Topic : 2 
 Words 0.000*"loopi" + 0.000*"implant" + 0.000*"recollect" + 0.000*"conic" + 0.000*"fish" + 0.000*"idnn" + 0.000*"temper" + 0.000*"symplect" + 0.000*"ramnet" + 0.000*"unwrap" 

Topic : 3 
 Words 0.000*"lrta" + 0.000*"sfmd" + 0.000*"kanji" + 0.000*"bandit" + 0.000*"ying" + 0.000*"chemotaxi" + 0.000*"lwpls" + 0.000*"seemor" + 0.000*"sasb" + 0.000*"caviti" 

Topic : 4 
 Words 0.000*"scalp" + 0.000*"intersegment" + 0.000*"mpcs" + 0.000*"lext" + 0.000*"trepan" + 0.000*"gemini" + 0.000*"pref" + 0.000*"boxlet" + 0.000*"chess" + 0.000*"massaro" 

Topic : 5 
 Words 0.000*"narx" + 0.000*"wind" + 0.000*"pact" + 0.000*"cholin

In [27]:
topic_dict = {}
for idx, topic in lda_tfidf.print_topics(-1):
    topic_dict[idx] = topic

In [28]:
for combination in combinations(list(range(10)), 2):
    common_words = set(topic_words(topic_dict[combination[0]])).intersection(set(topic_words(topic_dict[combination[1]])))
    print(f"Topic {combination[0]} and {combination[1]} : {len(common_words)}", end = "\n")
    print(f"Common words - {common_words}", end = "\n")
    print("\n")

Topic 0 and 1 : 0
Common words - set()


Topic 0 and 2 : 0
Common words - set()


Topic 0 and 3 : 0
Common words - set()


Topic 0 and 4 : 0
Common words - set()


Topic 0 and 5 : 0
Common words - set()


Topic 0 and 6 : 0
Common words - set()


Topic 0 and 7 : 0
Common words - set()


Topic 0 and 8 : 0
Common words - set()


Topic 0 and 9 : 0
Common words - set()


Topic 1 and 2 : 0
Common words - set()


Topic 1 and 3 : 1
Common words - {'caviti'}


Topic 1 and 4 : 0
Common words - set()


Topic 1 and 5 : 0
Common words - set()


Topic 1 and 6 : 0
Common words - set()


Topic 1 and 7 : 0
Common words - set()


Topic 1 and 8 : 0
Common words - set()


Topic 1 and 9 : 0
Common words - set()


Topic 2 and 3 : 0
Common words - set()


Topic 2 and 4 : 0
Common words - set()


Topic 2 and 5 : 0
Common words - set()


Topic 2 and 6 : 0
Common words - set()


Topic 2 and 7 : 0
Common words - set()


Topic 2 and 8 : 0
Common words - set()


Topic 2 and 9 : 0
Common words - set()


Topic 3 and

In [29]:
coherence_model_lda = CoherenceModel(model=lda_tfidf, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.6646907617821259


***
## **CONCLUSION: The topics have very little to none in common. We can conclude this because of the high coherence score and also because of the existence of very few common words b/w all pairs of topics.**
***